In [1]:
import pandas as pd
import numpy as np
import os
import itertools
import re

In [2]:
def listify(col):
    final = []
    for s in col:
        if s is np.NaN:
            continue
        else:
            final.append(s)
    string = str(final)
    for ch in ["[", "]", ",", "'", "nan"]:
        string = string.replace(ch, "")
    string = string.replace(u'\\xa0', u' ')
    return string

In [3]:
lab_slots = ['L1', 'L2', 'L3', 'L4', 'LX']

def slot_extract(value):
    final_slot = 0
    if value[0:1] is '':
        final_slot = 0
    else:
        try:
            final_slot = int(value[0:2].strip())
        except:
            gaatIt = 0
            for slot in lab_slots:
                if slot in value:
                    final_slot=slot
                    gaatIt = 1
                
                if gaatIt:
                    break
                

    return final_slot

In [4]:
def csv_to_df_gen(path_csv):
    return pd.read_csv(path_csv, encoding='cp1252')

In [5]:
def df_converter(df_dept):

    # drop useless cols
    cols_to_be_dropped = []
    for col in df_dept.columns:
        if (df_dept.isna().sum()[col]==df_dept.shape[0]):
            cols_to_be_dropped.append(col)

    df_dept = df_dept.drop(columns=cols_to_be_dropped)

    # drop useless rows
    row_toBe_dropped = []
    for row in df_dept.index:
        if (df_dept.isna().sum(axis=1)[row]==df_dept.shape[1]):
            row_toBe_dropped.append(row)
    df_dept=df_dept.drop(row_toBe_dropped)

    # correct indexing (bit redundant step)
    df_dept = df_dept.reset_index().drop(columns='index')

    # forward filling NaNs in Sr. No.
    df_dept[df_dept.columns[0]] = df_dept[df_dept.columns[0]].ffill()

    # getting final df
    df_final = pd.DataFrame(df_dept.groupby(df_dept.columns[0]).agg(listify))

    # processing final df
    df_final['Time'] = [s[2:] for s in df_final[df_dept.columns[6]]]
    df_final['Slot_code'] = [slot_extract(s) for s in df_final[df_dept.columns[6]]]
    df_final['Dept'] = [''.join(re.split('[^a-zA-Z]*', s)) for s in df_final[df_dept.columns[2]]]
    df_final['Yxx ka Y'] = [int(s[3:4]) for s in df_final[df_dept.columns[2]]]


    return df_final


In [6]:
def path_to_final_df_conv(dept_path):
    dept_df = csv_to_df_gen(dept_path)
    dept_df_final = df_converter(dept_df)
    return dept_df_final

In [7]:
def get_merged_df(path_list):
    df_list = []
    for path_dept in path_list:
        dept_csv_path = './' + path + '/' + path_dept
        df_list.append(path_to_final_df_conv(dept_csv_path))
    
    return pd.concat(df_list, ignore_index=True)

In [8]:
path = './dept_courses_csv'
dept_csv_names = os.listdir(path)
print(dept_csv_names)

['AE.csv', 'BB.csv', 'CE.csv', 'CH.csv', 'CL.csv', 'CM.csv', 'CSE.csv', 'DE.csv', 'EE.csv', 'EN.csv', 'ENT.csv', 'ES.csv', 'ET.csv', 'GNR.csv', 'GP.csv', 'HS.csv', 'IE.csv', 'MA.csv', 'ME.csv', 'MG.csv', 'MM.csv', 'PH.csv', 'PS.csv', 'SC.csv', 'TD.csv', 'US.csv']


In [9]:
# dept_csv_names1 = ['IE.csv', 'SC.csv', 'EE.csv', 'DE.csv', 'HS.csv']

df_merged = get_merged_df(dept_csv_names)
df_merged.shape

(798, 14)

In [20]:
# df_merged.to_csv('all_dept_merged.csv')

In [10]:
#Filters

In [11]:
def get_filtered_df(filters, filter_values, req_attr, df_all):
    
    df_filter_lst = []
    combs_not_found = []
    
    combs = list(itertools.product(*filter_values))

    for comb in combs:
        try:
            df_filter_lst.append(pd.DataFrame(df_all.groupby(filters).get_group(comb)[req_attr]))
        except:
            combs_not_found.append(comb)
    
    return pd.concat(df_filter_lst, ignore_index=True), combs_not_found

In [12]:
df_merged['Dept'].unique()

array(['AE', 'BB', 'CE', 'CH', 'CL', 'CM', 'CS', 'DE', 'EE', 'EN', 'ENT',
       'ES', 'ET', 'GNR', 'GP', 'GS', 'HS', 'IE', 'MA', 'SI', 'ME', 'MGT',
       'SOM', 'MM', 'PH', 'PS', 'SC', 'TD', 'US'], dtype=object)

In [13]:
filter_obo = [
    'Dept', 
    'Slot_code',
    'Yxx ka Y',
    ]

get_attr = [
    'Course Code',
    'Course Name', 
    'Dept', 
    'Division',
    'Instructor(s)',
    'Slot_code', 
    'Time', 
    'Yxx ka Y',
    ]

dept_list = ['CS', 'EE', 'ENT', 'ET', 'GNR', 'HS', 'IE', 'SI', 'MGT', 'SOM', 'MM', 'SC',]

slot_list = [6, 7, 12, 13, 14, 15]

year_list = [6, 7]

req_filter_value = [
    dept_list, 
    slot_list,
    year_list,
    ]

In [14]:
df_req_fil, inv_filt = get_filtered_df(filter_obo, req_filter_value, get_attr, df_merged)

In [15]:
print(len(inv_filt))

107


In [16]:
df_req_fil

,Course Code,Course Name,Dept,Division,Instructor(s),Slot_code,Time,Yxx ka Y
0,CS 620,New Trend in Information Technology,CS,,I - Supratik Chakraborty,12,Mon-12A-17:30:00-18:55:00 Thu-12B-17:30:00-18...,6
1,CS 748,Advances in Intelligent and Learning Agents,CS,,I - Shivaram Kalyanakrish,13,Mon-13A-19:00:00-20:25:00 Thu-13B-19:00:00-20...,7
2,CS 754,Advanced Image Processing,CS,,I - Ajit V. Rajwade,13,Mon-13A-19:00:00-20:25:00 Thu-13B-19:00:00-20...,7
3,CS 736,Medical Image Computing,CS,,I - Suyash P. Awate,14,Fri-14B-17:30:00-18:55:00 Tue-14A-17:30:00-18...,7
4,CS 741,Advanced Network Security and Cryptography,CS,,I - B. L. Menezes,14,Fri-14B-17:30:00-18:55:00 Tue-14A-17:30:00-18...,7
...,...,...,...,...,...,...,...,...
75,SC 702,Linear Systems Theory for PDEs,SC,M,I - Vivek Natarajan,6,Fri-6B-11:05:00-12:30:00 Wed-6A-11:05:00-12:30:00,7
76,SC 630,Variable Structure and Sliding Mode Control,SC,,I - Bandyopadhyay B.,13,Mon-13A-19:00:00-20:25:00 Thu-13B-19:00:00-20...,6
77,SC 630,Variable Structure and Sliding Mode Control,SC,M,I - Bandyopadhyay B.,13,Mon-13A-19:00:00-20:25:00 Thu-13B-19:00:00-20...,6
78,SC 602,Control of Nonlinear Dynamical Systems,SC,,I - Sukumar Srikant,14,Fri-14B-17:30:00-18:55:00 Tue-14A-17:30:00-18...,6


In [48]:
# len(list(df_merged['Dept'].unique()))

In [18]:
df_merged.to_csv('all_dept_course_list.csv')

In [19]:
df_merged.groupby(['Dept', 'Slot_code', 'Yxx ka Y']).get_group(('EE', 1, 1))
# df.groupby(filter_obo).get_group(('EE', 9, 7))[get_attr]

,Course Code,Course Name,Dept,Division,Division.1,Instructor(s),Registration Limit,Restrictions,Slot,Slot_code,Time,Type,Venue,Yxx ka Y
246,EE 114,Power Engineering - I,EE,S2,,I - Soman S.A.,0,NaN,1 Mon-1A-08:30:00-09:25:00 Thu-1C-10:35:00-11:...,1,Mon-1A-08:30:00-09:25:00 Thu-1C-10:35:00-11:30...,timetable,,1


In [273]:
df_merged.to_csv('merged_csv_1.csv')

In [246]:
df_merged['Dept'].unique()

array(['AE', 'BB', 'CE', 'CH', 'CS', 'DE', 'EE', 'EN', 'ET', 'GN', 'HS',
       'IE', 'MA', 'SI', 'MG', 'SO', 'SC'], dtype=object)

In [210]:
free_slots = [5, 6, 7, 12, 13, 14, 15]

for slot in free_slots:
    print('Slot: ' + str(slot))
    try:
        print((df_merged.groupby(['Slot_code', 'Yxx ka Y']).get_group((slot, 7))[['Course Code', 'Course Name']]))
    except:
        print('Either some error occured or No courses were found :((( ')
    print()

Slot: 5
   Course Code                                Course Name
76      EE 709  Testing and Verification of VLSI Circuits
84      EE 737         Introduction to Stochastic Control
96      EE 793                       Topics in Cryptology

Slot: 6
    Course Code                                      Course Name
14       IE 714  Quantitative Models for Supply Chain Management
33       SC 702                   Linear Systems Theory for PDEs
34       SC 702                   Linear Systems Theory for PDEs
97       EE 799                                Advanced Antennas
157      HS 710                              Consumer Psychology

Slot: 7
Either some error occured or No courses were found :((( 

Slot: 12
   Course Code                                       Course Name
79      EE 724                                   Nanoelectronics
92      EE 786  Developments Trends and Economic Frontiers in EE

Slot: 13
   Course Code                        Course Name
80      EE 727  Physics of Nan

In [161]:
(df_merged['Course Code'][15][0:2])

'IE'

In [226]:
df_merged.columns

Index(['Course Code', 'Course Name', 'Dept', 'Division', 'Division.1',
       'Instructor(s)', 'Registration Limit', 'Restrictions', 'Slot',
       'Slot_code', 'Time', 'Type', 'Venue', 'Yxx ka Y'],
      dtype='object')

In [237]:
col_map = {
    'Course Code': 'Course Code',
    'Course Name': 'Course Name',
    'Course Dept': 'Dept',
    'Course Division': 'Division',
    'Division.1': 'Division.1',
    'Instructor': 'Instructor(s)',
    'Reg Limit': 'Registration Limit',
    'Restrictions': 'Restrictions',
    'Course Slot': 'Slot',
    'Course Slot Code': 'Slot_code',
    'Course Timing': 'Time',
    'Course Type': 'Type',
    'Course Venue': 'Venue',
    'Course Year': 'Yxx ka Y', 
}

In [244]:
dept_list = ['EE', 'HS', 'MA', 'ME']
year_list = [4, 6, 7]
slot_list = [5, 3, 6]

filter_list = [dept_list, year_list, slot_list]
list(itertools.product(*filter_list))

[('EE', 4, 5),
 ('EE', 4, 3),
 ('EE', 4, 6),
 ('EE', 6, 5),
 ('EE', 6, 3),
 ('EE', 6, 6),
 ('EE', 7, 5),
 ('EE', 7, 3),
 ('EE', 7, 6),
 ('HS', 4, 5),
 ('HS', 4, 3),
 ('HS', 4, 6),
 ('HS', 6, 5),
 ('HS', 6, 3),
 ('HS', 6, 6),
 ('HS', 7, 5),
 ('HS', 7, 3),
 ('HS', 7, 6),
 ('MA', 4, 5),
 ('MA', 4, 3),
 ('MA', 4, 6),
 ('MA', 6, 5),
 ('MA', 6, 3),
 ('MA', 6, 6),
 ('MA', 7, 5),
 ('MA', 7, 3),
 ('MA', 7, 6),
 ('ME', 4, 5),
 ('ME', 4, 3),
 ('ME', 4, 6),
 ('ME', 6, 5),
 ('ME', 6, 3),
 ('ME', 6, 6),
 ('ME', 7, 5),
 ('ME', 7, 3),
 ('ME', 7, 6)]